# 最小限のFine-tuning実践ガイド（Google Colab）

このノートブックでは、最小限のリソースで効果的なfine-tuningを行う方法を説明します。
- **モデル**: TinyLlama 1.1B（軽量で高速）
- **手法**: QLoRA（メモリ効率的）
- **データ**: 50-100サンプル程度
- **実行時間**: 約10-20分

## 1. 環境セットアップ（約2分）

In [ ]:
# Unslothのインストール（最も簡単な方法）
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

## 2. データ準備（1分）
最小限のサンプルデータを作成します。

In [ ]:
import json

# 最小限のトレーニングデータ（50サンプル）
training_data = [
    {
        "instruction": "Pythonでリストを逆順にする方法を教えてください",
        "output": "Pythonでリストを逆順にする方法は3つあります：\n1. `reversed()`: `list(reversed(my_list))`\n2. スライス: `my_list[::-1]`\n3. `reverse()`: `my_list.reverse()`"
    },
    {
        "instruction": "gitでコミットを取り消す方法",
        "output": "gitでコミットを取り消すには：\n- `git reset --soft HEAD~1`: コミットのみ取り消し\n- `git reset --hard HEAD~1`: 変更ごと取り消し\n- `git commit --amend`: 直前のコミットを修正"
    },
    {
        "instruction": "Dockerコンテナが起動しない",
        "output": "Dockerコンテナが起動しない場合の確認事項：\n1. `docker logs <container_id>`: ログ確認\n2. `docker ps -a`: コンテナ状態確認\n3. Dockerfileのエントリーポイント確認\n4. ポート競合の確認"
    },
    {
        "instruction": "TypeError: 'NoneType' object is not iterable",
        "output": "このエラーは、Noneに対して繰り返し処理を行おうとした時に発生します。\n対処法：\n1. 変数がNoneでないか確認: `if data is not None:`\n2. デフォルト値を設定: `data = data or []`\n3. 関数の戻り値を確認"
    },
    {
        "instruction": "APIのレスポンスが遅い",
        "output": "APIレスポンスの改善方法：\n1. キャッシュの実装（Redis等）\n2. データベースクエリの最適化\n3. ページネーションの導入\n4. 非同期処理の活用\n5. CDNの利用"
    }
]

# データを増やす（バリエーションを追加）
extended_data = []
for item in training_data:
    # オリジナル
    extended_data.append(item)
    
    # 少し違う聞き方のバージョン
    variation = {
        "instruction": item["instruction"] + "。初心者向けに説明してください",
        "output": "初心者の方向けに説明します。\n\n" + item["output"]
    }
    extended_data.append(variation)

# 合計10個のサンプル（デモ用最小限）
print(f"トレーニングデータ数: {len(extended_data)}")

# JSONファイルとして保存
with open('minimal_training_data.json', 'w', encoding='utf-8') as f:
    json.dump(extended_data, f, ensure_ascii=False, indent=2)

## 3. モデルの読み込みとLoRA設定（2分）

In [ ]:
from unsloth import FastLanguageModel
import torch

# 最大シーケンス長（短めに設定してメモリ節約）
max_seq_length = 512

# 4bit量子化で1.1Bモデルを読み込み（約500MB）
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/tinyllama-bnb-4bit",  # 最軽量モデル
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)

# LoRAアダプターを追加（メモリ効率的なfine-tuning）
model = FastLanguageModel.get_peft_model(
    model,
    r=8,  # ランク（小さくしてメモリ節約）
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)

print("モデルの準備完了！")

## 4. データセットの準備（1分）

In [ ]:
from datasets import Dataset

# プロンプトフォーマット関数
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["output"]
    texts = []
    
    for instruction, output in zip(instructions, outputs):
        text = f"""### 質問:
{instruction}

### 回答:
{output}"""
        texts.append(text)
    
    return {"text": texts}

# データセット作成
dataset = Dataset.from_list(extended_data)
print(f"データセットサイズ: {len(dataset)}")

## 5. Fine-tuning実行（10-15分）

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# トレーナー設定（最小限）
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    formatting_func=formatting_prompts_func,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        warmup_steps=5,
        max_steps=30,  # 最小限のステップ数
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        seed=3407,
        output_dir="outputs",
    ),
)

# GPUメモリ統計
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"使用中のメモリ = {start_gpu_memory} GB.")

# トレーニング開始
trainer_stats = trainer.train()

# メモリ使用量
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"トレーニングで使用したメモリ = {used_memory_for_lora} GB ({lora_percentage}% of GPU)")

## 6. モデルのテスト（1分）

In [ ]:
# 推論モードに切り替え
FastLanguageModel.for_inference(model)

# テスト質問
test_questions = [
    "Pythonでファイルを読み込む方法は？",
    "エラーが出ました: KeyError: 'user_id'",
    "gitで変更を取り消したい"
]

for question in test_questions:
    inputs = tokenizer(
        f"""### 質問:
{question}

### 回答:""",
        return_tensors="pt"
    ).to("cuda")
    
    outputs = model.generate(**inputs, max_new_tokens=128, temperature=0.7)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    print(f"\n質問: {question}")
    print(f"回答: {response.split('### 回答:')[1].strip()}")
    print("-" * 50)

## 7. GGUF形式で保存（Ollama用）（2分）

In [ ]:
# GGUF形式で保存（量子化レベル: q4_k_m）
print("GGUF形式で保存中...")
model.save_pretrained_gguf(
    "minimal_finetuned_model",
    tokenizer,
    quantization_method="q4_k_m"  # バランスの良い量子化
)

print("\n✅ 保存完了！")
print("ファイル名: minimal_finetuned_model-unsloth.Q4_K_M.gguf")
print("\nこのファイルをダウンロードして、Ollamaで使用できます。")

## 8. ファイルのダウンロード

In [ ]:
# Google Colabからファイルをダウンロード
from google.colab import files

# GGUFファイルをダウンロード
files.download('minimal_finetuned_model-unsloth.Q4_K_M.gguf')

print("\n📥 ファイルのダウンロードが開始されました。")
print("\nローカルでの使用方法:")
print("1. ダウンロードしたGGUFファイルを適当なディレクトリに配置")
print("2. Modelfileを作成:")
print("   echo 'FROM ./minimal_finetuned_model-unsloth.Q4_K_M.gguf' > Modelfile")
print("3. Ollamaでモデルを作成:")
print("   ollama create my-minimal-model -f Modelfile")
print("4. 使用:")
print("   ollama run my-minimal-model \"質問\"")

## まとめ

このノートブックでは、最小限のリソースでfine-tuningを実行しました：

- **モデル**: TinyLlama 1.1B（最軽量）
- **データ**: 10サンプル（実用では50-100推奨）
- **実行時間**: 約15-20分
- **GPU使用量**: 約3-4GB
- **出力**: Ollama対応のGGUFファイル

### 改善のヒント

1. **データを増やす**: 50-100サンプルで品質向上
2. **ステップ数を増やす**: max_steps=50-100
3. **大きめのモデル**: Llama-3.2-3B（余裕があれば）
4. **評価の追加**: 検証データセットで品質確認